In [1]:
%pylab inline

import os
from operator import concat

Populating the interactive namespace from numpy and matplotlib


In [2]:
filters = dict(
    CaHK=[2],
    CaII=range(47,50),
    Zfea=range(38,56),
    wall=range(56)
)
snr_hdr = map(concat, ["snr_"]*len(filters), filters.keys())
npx_hdr = map(concat, ["npix_"]*len(filters), filters.keys())
fields = ["input_file"] + sorted(snr_hdr) + sorted(npx_hdr)
print fields

['input_file', 'snr_CaHK', 'snr_CaII', 'snr_Zfea', 'snr_wall', 'npix_CaHK', 'npix_CaII', 'npix_Zfea', 'npix_wall']


In [3]:
idir = os.path.expandvars("$master/inputs/real_deal/nem_jpas/")
flist = sorted([file for root, subs, files in os.walk(idir) for file in files if file.startswith("processed_")])

input_n = []
snr, npx = dict(zip(filters.keys(),([],[],[],[]))), dict(zip(filters.keys(),([],[],[],[])))
for file in flist:
    fl, sg = loadtxt(idir+file, usecols=(1,2), unpack=True)

    input_n.append("spSpec"+file.lstrip("dynbasfit_processed_nem_JPAS_").replace(".txt", ".fit"))
    for kw in filters.keys():
        not_masked = fl[filters[kw]]>0.0
        
        snr[kw] += [mean((fl/sg)[filters[kw]][not_masked]) if any(not_masked) else 0.0]
        npx[kw] += [count_nonzero(not_masked)]

/home/mejia/Programs/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:13: RuntimeWarning: invalid value encountered in divide


In [4]:
oname = os.path.expandvars("$master/notebooks/data/RealDeal/nfilters.txt")
fmt = "%25s"+"%10.2f"*4+"%10d"*4
hdr = ("%23s"+"%10s"*4+"%10s"*4)%tuple(fields)

table = zeros(len(input_n), dtype=zip(fields,("|25S "+">i8 "*4+">f8 "*4).split()))
table["input_file"] = input_n
for kw in filters.keys():
    table["snr_"+kw] = snr[kw]
    table["npix_"+kw] = npx[kw]

savetxt(oname, table, fmt=fmt, header=hdr)